In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

In [2]:
def read_float32(fileIn):
    with open(fileIn, 'rb') as f:
        data = np.fromfile(f, dtype=np.dtype('>f'))
        print(np.shape(data))
    return data

def read_float64(fileIn):
    with open(fileIn, 'rb') as f:
        data = np.fromfile(f, dtype=np.dtype('>f8'))
        print(np.shape(data))
    return data


## Regional

In [3]:
dir = '/scratch/shoshi/MITgcm_c68p/regional_obsfit/run_obsfit_pk0000166559_F/'

tile1 = read_float64(dir + 'dummy_obsfit_etan.001.001.equi.data')
tile1

(4,)


array([-2.96776714,  1.        ,  1.        ,  1.        ])

In [4]:
etan = xr.open_dataset(dir + 'dummy_obsfit_etan.equi.nc')
mod_e = etan.mod_val.values
mod_e

array([-2.96776714])

In [11]:
(.01 - mod_e)**2

array([8.86709716])

## Global LLC90

In [39]:
dir = '/scratch/shoshi/MITgcm_c68p/llc90/run_obsfit_pk0000205956_T/'

data = xr.open_dataset(dir + 'obsfit_SSH.nc')

obs_date = data['obs_YYYYMMDD']
mask = obs_date == 20150701
data_0701 = data.where(mask, drop=True)
data_0701

obs_time = data_0701['obs_HHMMSS']
mask = obs_time < 20000
data_intime = argo_0701.where(mask, drop=True)
# data_intime = data.sel(obs_YYYYMMDD=20150701)
#data_intime

data_val = data_intime.obs_val.values
len(data_val[~np.isnan(data_val)])

data

<xarray.Dataset>
Dimensions:               (iOBS: 13297, iSAMPLE: 13297, iINTERP: 8)
Dimensions without coordinates: iOBS, iSAMPLE, iINTERP
Data variables: (12/16)
    obs_date              (iOBS) float64 ...
    obs_YYYYMMDD          (iOBS) float64 2.015e+07 2.015e+07 ... 2.015e+07
    obs_HHMMSS            (iOBS) float64 ...
    sample_x              (iSAMPLE) float64 ...
    sample_y              (iSAMPLE) float64 ...
    sample_z              (iSAMPLE) float64 ...
    ...                    ...
    sample_interp_YC11    (iSAMPLE) float64 ...
    sample_interp_XCNINJ  (iSAMPLE) float64 ...
    sample_interp_YCNINJ  (iSAMPLE) float64 ...
    sample_interp_i       (iSAMPLE) float64 ...
    sample_interp_j       (iSAMPLE) float64 ...
    sample_interp_w       (iSAMPLE, iINTERP) float64 ...

In [40]:
# 23 model equivalent values, but 6 are in blanklist

equi = xr.open_dataset(dir + 'obsfit_SSH.equi.nc')
#mod_e = equi.mod_mask.values * equi.mod_val.values
mod_e = equi.mod_val.values
len(mod_e[~np.isnan(mod_e)])
equi

<xarray.Dataset>
Dimensions:   (iOBS: 13297)
Dimensions without coordinates: iOBS
Data variables:
    mod_val   (iOBS) float64 -0.01729 0.229 nan 0.4538 ... nan nan nan nan
    mod_mask  (iOBS) float64 ...

In [42]:
np.nansum((data.obs_val - equi.mod_val)**2)

8810.060708183351

## Global LLC90 just 3

In [33]:
dir = '/scratch/shoshi/MITgcm_c68p/llc90/run_obsfit_pk0000205956_Q/'

data = xr.open_dataset(dir + 'obsfit_SSH.nc')
data.obs_val.values

array([        nan, 25.81075603, 30.43173803, ..., 30.00871879,
               nan, 27.62982594])

In [34]:
#np.nansum((equi.mod_val.values - etan.obs_val.values)**2)
model = xr.open_dataset(dir + 'obsfit_SSH.equi.nc')
model.mod_val.values, model.mod_mask.values

(array([0.68978245, 1.20940546, 0.09422804]), array([1., 1., 1.]))

In [32]:
np.nansum((model.mod_val.values - data.obs_val.values)**2)

8810.060708183351

In [20]:
tile1 = read_float64(dir + 'obsfit_SSH.011.001.equi.data')
tile2 = read_float64(dir + 'obsfit_SSH.029.001.equi.data')
tile3 = read_float64(dir + 'obsfit_SSH.037.001.equi.data')

tile1, tile2, tile3

(4,)
(4,)
(4,)


(array([-0.01553957,  1.        ,  1.        ,  1.        ]),
 array([0.6055279, 2.       , 1.       , 2.       ]),
 array([1.20940546, 3.        , 1.        , 3.        ]))

In [163]:
d = {'obs_data': [np.nan, 0.25810756E+02, 0.30431738E+02], 
     'obs_weight': [0.10000000E+01, 0.10000000E+01, 0.10000000E+01],
    'obs_modmask': [0.10000000E+01, 0.10000000E+01, 0.10000000E+01],
    'obs_modval': [-0.15539569E-01, 0.60552790E+00, 0.12094055E+01]}

df = pd.DataFrame(data=d)
df

,obs_data,obs_weight,obs_modmask,obs_modval
0,NaN,1.0,1.0,-0.015540
1,25.810756,1.0,1.0,0.605528
2,30.431738,1.0,1.0,1.209405


In [165]:
np.sum((df['obs_data'] - df['obs_modval'])**2)

1489.248240313586